# [Rotting Oranges](https://leetcode.com/problems/rotting-oranges/)
- **Given**: A grid `M`x`N`
    + 0: empty cell
    + 1: fresh orange
    + 2: rotten orange
- Every minute, any **fresh** orange that is 4-directionally adjacent to a **rotten** orange becomes rotten.
- **Return**: the minimum number of minutes that must elapse until no cell has a fresh orange
    + If this is impossible, return `-1`

#### Example 1

```
Input: grid = [
    [2,1,1],
    [1,1,0],
    [0,1,1]]
Output: 4
Explanation:
```

<img src="./assets/3.png" width="1000"/>

#### Example 2, 3

```
Input: grid = [
    [2,1,1],
    [0,1,1],
    [1,0,1]]
Output: -1
Explanation: The orange in the bottom left corner (row 2, column 0) is never rotten, because rotting only happens 4-directionally.


Input: grid = [[0,2]]
Output: 0
Explanation: Since there are already no fresh oranges at minute 0, the answer is just 0.
```

#### Constraints
- $1 \leq M, N \leq 10$

## Solution 1: Floodfill BFS

```C++
class Solution {
public:
    int orangesRotting(vector<vector<int>> &grid) {
        int N = grid.size();
        if(N==0) return 0;
        int M = grid[0].size();

        // Find all rotten oranges
        vector<pair<int,int>> rottens;
        int fresh_cnt = 0;
        for(int i=0; i<N; ++i) for(int j=0; j<M; ++j) {
            if(grid[i][j] == 1) {
                fresh_cnt += 1;
            }
            if(grid[i][j] == 2) {
                rottens.push_back( {i,j} );
            }
        }
        if(rottens.size() == 0 && fresh_cnt == 0) return 0;

        // BFS: floodfill
        int front = 0;
        int minutes = -1;
        while(front < rottens.size()) {
            minutes += 1;
            int batch_size = rottens.size();
            for(int rotten_idx=front; rotten_idx < batch_size; ++rotten_idx) {
                int i = rottens[rotten_idx].first;
                int j = rottens[rotten_idx].second;
                front += 1;


                for(auto &[i_, j_]: vector<pair<int,int>>{{0,1}, {0,-1}, {1,0}, {-1, 0}}) {
                    int i_next = i + i_;
                    int j_next = j + j_;

                    if(0 <= i_next && i_next < N && 
                            0 <= j_next && j_next < M && 
                            grid[i_next][j_next] == 1) {
                        grid[i_next][j_next] = 2;
                        rottens.push_back( {i_next, j_next} );
                        fresh_cnt -= 1;
                    }
                }
            }
        }
        return (fresh_cnt == 0) ? minutes : -1;
    }
};
```

## Solution 2: BFS with Priority Queue

```C++
class Solution {
public:
    int orangesRotting(vector<vector<int>>& grid) {
        int N = grid.size();
        if(N==0) return 0;
        int M = grid[0].size();

        // Find all rottens
        multimap<int, pair<int, int>> PQ;
        int fresh_cnt = 0;
        for(int i=0; i<N; ++i) for(int j=0; j<M; ++j) {
            if(grid[i][j] == 1) {
                fresh_cnt += 1;
            }
            if(grid[i][j] == 2) {
                PQ.insert( {0, {i,j}} );
            }
        }

        // BFS with priority queue (less time/minute)
        int ans = 0;
        while(!PQ.empty()) {
            int minute = PQ.begin()->first;
            int i = PQ.begin()->second.first;
            int j = PQ.begin()->second.second;
            PQ.erase(PQ.lower_bound(minute));

            ans = max(ans, minute);

            for(auto &[i_, j_]: vector<pair<int,int>>{{0,1}, {0,-1}, {1,0}, {-1, 0}}) {
                int i_next = i + i_;
                int j_next = j + j_;

                if(0 <= i_next && i_next< N &&
                        0 <= j_next && j_next< M &&
                        grid[i_next][j_next] == 1) {
                    grid[i_next][j_next] = 2;
                    PQ.insert( {minute+1, {i_next,j_next}} );
                    fresh_cnt -= 1;
                }
            }
        }
        return (fresh_cnt == 0) ? ans : -1;
    }
};
```